# Project 4 Calculating the energy usage index (Energy Consumption (per day )divided by floor area (square foot) for a given UC Berkley building

In [1]:
!pwd

/home/jovyan/sp21/proj/proj5


In [2]:
# Run this cell to set up imports
import numpy as np
import pandas as pd

In [3]:
%reload_ext sql
%sql postgresql://jovyan@127.0.0.1:5432/template1

In [4]:
import subprocess
import os
import warnings

call = subprocess.run(["psql", "-h", "localhost", \
                       "-tAc", "SELECT 1 FROM pg_database WHERE datname='ucb_buildings'", "template1"], \
                      stdout=subprocess.PIPE, text=True)

if call.stdout != "1\n":
    os.system("gunzip -c proj3.sql.gz | psql -h localhost -d template1 -f -")
else:
    warnings.warn("you need to run dropdb -h localhost ucb_buildings if you want to reload the database.")
%sql postgresql://jovyan@127.0.0.1:5432/ucb_buildings

<ipython-input-4-70667cec2ae0>:12: UserWarning: you need to run dropdb -h localhost ucb_buildings if you want to reload the database.
  warnings.warn("you need to run dropdb -h localhost ucb_buildings if you want to reload the database.")


# SQL model No.1 - select the bulding that has the largest amount of data 

In [5]:
%%sql
DROP TABLE IF EXISTS bldg;
CREATE TABLE bldg AS(
SELECT site, count(value)
FROM metadata, data
WHERE metadata.id = data.id
GROUP BY site
ORDER BY count desc
LIMIT 1);
SELECT * FROM bldg

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
Done.
1 rows affected.
1 rows affected.


site,count
Wurster Hall NEW (as of 12/6/10),11999


# SQL model No.2 - select the bulding site and map it to buildings_site_mapping

In [6]:
%%sql
DROP TABLE IF EXISTS bldg_site;
CREATE TABLE bldg_site AS(
SELECT buildings_site_mapping.*
FROM buildings_site_mapping, building
WHERE buildings_site_mapping.site = building.site);
SELECT * FROM bldg_site

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
Done.
(psycopg2.errors.UndefinedTable) relation "building" does not exist
LINE 3: FROM buildings_site_mapping, building
                                     ^

[SQL: CREATE TABLE bldg_site AS(
SELECT buildings_site_mapping.*
FROM buildings_site_mapping, building
WHERE buildings_site_mapping.site = building.site);]
(Background on this error at: http://sqlalche.me/e/14/f405)


# SQL model No.3 - select the building name and map it to real_eastate_metadata

In [7]:
%%sql
DROP TABLE IF EXISTS bldg_real_estate;
CREATE TABLE bldg_real_estate AS(
SELECT rem.*
FROM bldg_site bs
INNER JOIN real_estate_metadata rem ON bs.building = rem.building_name);
SELECT * FROM bldg_real_estate

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
Done.
(psycopg2.errors.UndefinedTable) relation "bldg_site" does not exist
LINE 3: FROM bldg_site bs
             ^

[SQL: CREATE TABLE bldg_real_estate AS(
SELECT rem.*
FROM bldg_site bs
INNER JOIN real_estate_metadata rem ON bs.building = rem.building_name);]
(Background on this error at: http://sqlalche.me/e/14/f405)


# SQL model No.4 - get the metadata of by building site

In [8]:
%%sql
DROP TABLE IF EXISTS bldg_metadata;
CREATE TABLE bldg_metadata AS(
SELECT metadata.*
FROM metadata
INNER JOIN bldg_site bs ON bs.site=metadata.site);
SELECT * FROM bldg_metadata LIMIT 5

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
Done.
(psycopg2.errors.UndefinedTable) relation "bldg_site" does not exist
LINE 4: INNER JOIN bldg_site bs ON bs.site=metadata.site);
                   ^

[SQL: CREATE TABLE bldg_metadata AS(
SELECT metadata.*
FROM metadata
INNER JOIN bldg_site bs ON bs.site=metadata.site);]
(Background on this error at: http://sqlalche.me/e/14/f405)


# SQL model No.5 - get energy related metadata by units

In [9]:
%%sql
DROP TABLE IF EXISTS bldg_energy_metadata;
CREATE TABLE bldg_energy_metadata AS(
SELECT * 
FROM bldg_metadata
WHERE units in ('kwh', 'Kwh', 'kWh', 'kwH', 'KWh', 'KwH', 'kWH', 'KWH', 'kw', 'Kw', 'kW', 'KW'));
SELECT * FROM bldg_energy_metadata LIMIT 5

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
Done.
(psycopg2.errors.UndefinedTable) relation "bldg_metadata" does not exist
LINE 3: FROM bldg_metadata
             ^

[SQL: CREATE TABLE bldg_energy_metadata AS(
SELECT * 
FROM bldg_metadata
WHERE units in ('kwh', 'Kwh', 'kWh', 'kwH', 'KWh', 'KwH', 'kWH', 'KWH', 'kw', 'Kw', 'kW', 'KW'));]
(Background on this error at: http://sqlalche.me/e/14/f405)


# SQL Model No.6 - get energy related class metadata, group by

In [10]:
%%sql
DROP TABLE IF EXISTS bldg_energy_raw_classes;
CREATE TABLE bldg_energy_raw_classes AS(
SELECT class
FROM bldg_energy_metadata
GROUP BY class);
SELECT * FROM bldg_energy_raw_classes LIMIT 5

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
Done.
(psycopg2.errors.UndefinedTable) relation "bldg_energy_metadata" does not exist
LINE 3: FROM bldg_energy_metadata
             ^

[SQL: CREATE TABLE bldg_energy_raw_classes AS(
SELECT class
FROM bldg_energy_metadata
GROUP BY class);]
(Background on this error at: http://sqlalche.me/e/14/f405)


# SQL Model No.7 - get brick class through mapping the raw class name to the mapping table 
(may be we can imporve entity resolution of rawnames the before this step)

In [11]:
%%sql
DROP TABLE IF EXISTS bldg_energy_brick_classes;
CREATE TABLE bldg_energy_brick_classes AS(
SELECT m.*
FROM bldg_energy_raw_classes berc INNER JOIN mapping m
ON berc.class = m.rawname);
SELECT * FROM bldg_energy_brick_classes ORDER BY brickclass

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
Done.
(psycopg2.errors.UndefinedTable) relation "bldg_energy_raw_classes" does not exist
LINE 3: FROM bldg_energy_raw_classes berc INNER JOIN mapping m
             ^

[SQL: CREATE TABLE bldg_energy_brick_classes AS(
SELECT m.*
FROM bldg_energy_raw_classes berc INNER JOIN mapping m
ON berc.class = m.rawname);]
(Background on this error at: http://sqlalche.me/e/14/f405)


# SQL Model No.8 - get the raw class with Energy_Sensor brickclass

In [12]:
%%sql
DROP TABLE IF EXISTS bldg_energy_classes;
CREATE TABLE bldg_energy_classes AS(
SELECT rawname 
FROM bldg_energy_brick_classes
WHERE brickclass = 'https://brickschema.org/schema/Brick#Energy_Sensor');
SELECT * FROM bldg_energy_classes

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
Done.
(psycopg2.errors.UndefinedTable) relation "bldg_energy_brick_classes" does not exist
LINE 3: FROM bldg_energy_brick_classes
             ^

[SQL: CREATE TABLE bldg_energy_classes AS(
SELECT rawname 
FROM bldg_energy_brick_classes
WHERE brickclass = 'https://brickschema.org/schema/Brick#Energy_Sensor');]
(Background on this error at: http://sqlalche.me/e/14/f405)


# SQL Model No.9 - get data of the energy class

In [13]:
%%sql
DROP TABLE IF EXISTS bldg_energy_data;
CREATE TABLE bldg_energy_data AS(
SELECT data.*, bldg_energy_metadata.class 
FROM bldg_energy_metadata, bldg_energy_classes, data
WHERE bldg_energy_metadata.class = bldg_energy_classes.rawname and bldg_energy_metadata.id = data.id);
SELECT * FROM bldg_energy_data LIMIT 5

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
Done.
(psycopg2.errors.UndefinedTable) relation "bldg_energy_metadata" does not exist
LINE 3: FROM bldg_energy_metadata, bldg_energy_classes, data
             ^

[SQL: CREATE TABLE bldg_energy_data AS(
SELECT data.*, bldg_energy_metadata.class 
FROM bldg_energy_metadata, bldg_energy_classes, data
WHERE bldg_energy_metadata.class = bldg_energy_classes.rawname and bldg_energy_metadata.id = data.id);]
(Background on this error at: http://sqlalche.me/e/14/f405)


# SQL Model No.10 - get the most largest kind of energy data

In [14]:
%%sql
DROP TABLE IF EXISTS bldg_energy_data_sum_class;
CREATE TABLE bldg_energy_data_sum_class AS(
SELECT class
FROM bldg_energy_data
ORDER BY value DESC
LIMIT 1);
SELECT * FROM bldg_energy_data_sum_class

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
Done.
(psycopg2.errors.UndefinedTable) relation "bldg_energy_data" does not exist
LINE 3: FROM bldg_energy_data
             ^

[SQL: CREATE TABLE bldg_energy_data_sum_class AS(
SELECT class
FROM bldg_energy_data
ORDER BY value DESC
LIMIT 1);]
(Background on this error at: http://sqlalche.me/e/14/f405)


# SQL Model No.11 - get the energy comsuption data of the day 2018-06-10
(cleaning outliers of temp sensor data (pj3, q2))

In [15]:
%%sql
DROP TABLE IF EXISTS bldg_energy_consumption;
CREATE TABLE bldg_energy_consumption AS(
SELECT max(bed.value) - min(bed.value) as energy_consumption_kWh
FROM bldg_energy_data bed, bldg_energy_data_sum_class bedsc
WHERE bed.class = bedsc.class and bed.time >= '2018-06-10' and bed.time < '2018-06-11');
SELECT * FROM bldg_energy_consumption

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
Done.
(psycopg2.errors.UndefinedTable) relation "bldg_energy_data" does not exist
LINE 3: FROM bldg_energy_data bed, bldg_energy_data_sum_class bedsc
             ^

[SQL: CREATE TABLE bldg_energy_consumption AS(
SELECT max(bed.value) - min(bed.value) as energy_consumption_kWh
FROM bldg_energy_data bed, bldg_energy_data_sum_class bedsc
WHERE bed.class = bedsc.class and bed.time >= '2018-06-10' and bed.time < '2018-06-11');]
(Background on this error at: http://sqlalche.me/e/14/f405)


# SQL Model No.12 - calculate the Energy Usage Index (EUI) of that bulding of that day

In [16]:
%%sql
DROP TABLE IF EXISTS bldg_eui;
CREATE TABLE bldg_eui AS(
SELECT bec.energy_consumption_kwh / bre.bldg_asf as EUI_kWh_sf
FROM bldg_energy_consumption bec, bldg_real_estate bre);
SELECT * FROM bldg_eui

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
Done.
(psycopg2.errors.UndefinedTable) relation "bldg_energy_consumption" does not exist
LINE 3: FROM bldg_energy_consumption bec, bldg_real_estate bre);
             ^

[SQL: CREATE TABLE bldg_eui AS(
SELECT bec.energy_consumption_kwh / bre.bldg_asf as EUI_kWh_sf
FROM bldg_energy_consumption bec, bldg_real_estate bre);]
(Background on this error at: http://sqlalche.me/e/14/f405)
